In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim




In [2]:
###Import dataset

input_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_inputV2_for_NN_20210514_to_20240404.tsv'

# Read the tab-separated file
# The delimiter '\t' specifies that the fields are separated by tabs
input_dataset = pd.read_csv(input_file, delimiter='\t')


# Display the first few rows of the DataFrame
print(input_dataset.iloc[:5,:5])
print(input_dataset.shape)

                                                    A_20210723  A_20210726  \
EOD_option_call_total_volume_strike_NAto0.5_exp...       0.392       0.386   
EOD_option_call_total_open_interest_strike_NAto...       0.347       0.340   
EOD_option_put_total_volume_strike_NAto0.5_expi...       0.389       0.382   
EOD_option_put_total_open_interest_strike_NAto0...       0.245       0.238   
EOD_option_call_total_volume_strike_NAto0.5_exp...       0.393       0.387   

                                                    A_20210727  A_20210728  \
EOD_option_call_total_volume_strike_NAto0.5_exp...       0.381       0.353   
EOD_option_call_total_open_interest_strike_NAto...       0.334       0.306   
EOD_option_put_total_volume_strike_NAto0.5_expi...       0.377       0.350   
EOD_option_put_total_open_interest_strike_NAto0...       0.234       0.208   
EOD_option_call_total_volume_strike_NAto0.5_exp...       0.382       0.354   

                                                    A_20210729

In [36]:
# Ensure reproducibility
np.random.seed(42)

test_10d_change_file  = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_change10d_for_NN_20210514_to_20240404.tsv'
change10d_dataset = pd.read_csv(test_10d_change_file, delimiter='\t')
print(change10d_dataset.iloc[:5,:5])
print(change10d_dataset.shape)

neg_output_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_5down10d_for_NN_20210514_to_20240404.tsv'
neg_output_dataset = pd.read_csv(neg_output_file, delimiter='\t')
print(neg_output_dataset.iloc[:5,:5])
print(neg_output_dataset.shape)
print(neg_output_dataset.value_counts())

output9up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_9up10d_for_NN_20210514_to_20240404.tsv'
output9up10d_dataset = pd.read_csv(output9up10d_file, delimiter='\t')
print(output9up10d_dataset.iloc[:5,:5])
print(output9up10d_dataset.shape)
print(output9up10d_dataset.value_counts())

output7up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_7up10d_for_NN_20210514_to_20240404.tsv'
output7up10d_dataset = pd.read_csv(output7up10d_file, delimiter='\t')
print(output7up10d_dataset.iloc[:5,:5])
print(output7up10d_dataset.shape)
print(output7up10d_dataset.value_counts())

output5up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_5up10d_for_NN_20210514_to_20240404.tsv'
output5up10d_dataset = pd.read_csv(output5up10d_file, delimiter='\t')
print(output5up10d_dataset.iloc[:5,:5])
print(output5up10d_dataset.shape)
print(output5up10d_dataset.value_counts())

# Shuffle column names
shuffled_columns = np.random.permutation(input_dataset.columns)

# Calculate split sizes
total_columns = len(input_dataset.columns)
train_size = int(0.6 * total_columns)
cv_size = int(0.2 * total_columns)
# The remaining columns will go to the test set

# Split the columns
train_columns = shuffled_columns[:train_size]
cv_columns = shuffled_columns[train_size:train_size+cv_size]
test_columns = shuffled_columns[train_size+cv_size:]

# Create subsets of the DataFrame based on the columns
train_df = input_dataset[train_columns]
cv_df = input_dataset[cv_columns]
test_df = input_dataset[test_columns]

output9up10d_dataset.index = input_dataset.columns
output7up10d_dataset.index = input_dataset.columns
output5up10d_dataset.index = input_dataset.columns
neg_output_dataset.index = input_dataset.columns
change10d_dataset.index = input_dataset.columns

train9up10d_output = output9up10d_dataset.T[train_columns].T
cv9up10d_output = output9up10d_dataset.T[cv_columns].T
test9up10d_output = output9up10d_dataset.T[test_columns].T

train7up10d_output = output7up10d_dataset.T[train_columns].T
cv7up10d_output = output7up10d_dataset.T[cv_columns].T
test7up10d_output = output7up10d_dataset.T[test_columns].T

train5up10d_output = output5up10d_dataset.T[train_columns].T
cv5up10d_output = output5up10d_dataset.T[cv_columns].T
test5up10d_output = output5up10d_dataset.T[test_columns].T

train_negoutput = neg_output_dataset.T[train_columns].T
cv_negoutput = neg_output_dataset.T[cv_columns].T
test_negoutput = neg_output_dataset.T[test_columns].T

train_10dChange = change10d_dataset.T[train_columns].T
cv_10dChange = change10d_dataset.T[cv_columns].T
test_10dChange = change10d_dataset.T[test_columns].T

train9up10d_output.index = range(1, len(train9up10d_output) + 1)
cv9up10d_output.index = range(1, len(cv9up10d_output) + 1)
test9up10d_output.index = range(1, len(test9up10d_output) + 1)

train7up10d_output.index = range(1, len(train7up10d_output) + 1)
cv7up10d_output.index = range(1, len(cv7up10d_output) + 1)
test7up10d_output.index = range(1, len(test7up10d_output) + 1)

train5up10d_output.index = range(1, len(train5up10d_output) + 1)
cv5up10d_output.index = range(1, len(cv5up10d_output) + 1)
test5up10d_output.index = range(1, len(test5up10d_output) + 1)

train_negoutput.index = range(1, len(train_negoutput) + 1)
cv_negoutput.index = range(1, len(cv_negoutput) + 1)
test_negoutput.index = range(1, len(test_negoutput) + 1)

train_10dChange.index = range(1, len(train_10dChange) + 1)
cv_10dChange.index = range(1, len(cv_10dChange) + 1)
test_10dChange.index = range(1, len(test_10dChange) + 1)


         x
1  1.02226
2  1.04186
3  1.04381
4  1.03341
5  1.03589
(316022, 1)
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    234112
1     81910
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    286620
1     29402
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    265159
1     50863
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    229013
1     87009
dtype: int64


In [37]:
X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_negoutput), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act_output(self.output(x))
        return x
 
negmodel = PimaClassifier().to(device)
print(negmodel)

pos_weight = torch.tensor([2.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(negmodel.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = negmodel(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': negmodel.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the negmodel
neg_predictions = (test_model(X) > 0.5).int().to('cpu')
neg_predictions = neg_predictions.numpy()
test_pred = pd.DataFrame(neg_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_negoutput
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for negative predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

neg_predictions = 1 - neg_predictions

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model9up10d = PimaClassifier().to(device)
print(model9up10d)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model9up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model9up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model9up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model7up10d = PimaClassifier().to(device)
print(model7up10d)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model7up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model7up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model7up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model5up10d = PimaClassifier().to(device)
print(model5up10d)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model5up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model5up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model5up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


Using device: cuda
PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)
Finished epoch 0, latest loss 0.9713118672370911
Saved improved model
Finished epoch 1, latest loss 0.9539806842803955
Saved improved model
Finished epoch 2, latest loss 0.9343343377113342
Saved improved model
Finished epoch 3, latest loss 0.9297536015510559
Saved improved model
Finished epoch 4, latest loss 0.9281354546546936
Saved improved model
Finished epoch 5, latest loss 0.9215038418769836
Saved improved model
Finished epoch 6, latest loss 0.9096908569335938
Saved improved model
Finished epoch 7, latest loss 0.9031732678413391
Saved improved model
Finished epoch 8, latest loss 0.8996759057044983
Saved improved model
Fi

KeyboardInterrupt: 

In [28]:
# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

negmodel = PimaClassifier().to(device)
negcheckpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
negmodel.load_state_dict(negcheckpoint['model_state_dict'])
optimizer.load_state_dict(negcheckpoint['optimizer_state_dict'])
epoch = negcheckpoint['epoch']
loss = negcheckpoint['loss']
negmodel.eval()
negmodel = negmodel.to(device)

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(neg_predictions))
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 9up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 9up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 9up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 7up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 7up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 7up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 5up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 5up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 5up10d predictions:", mean_value)



63204
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1698                4116
Actual Negative                 524               56866
Positive predictive power:
29.21%
Positive predictive accuracy:
76.42%
Confusion Matrix for merged 9up10d predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1443                4371
Actual Negative                 342               57048
Positive predictive power for 9up10d:
24.82%
Positive predictive accuracy for 9up10d:
80.84%
Mean change for incorrect 9up10d predictions: 1.034
Mean change for correct 9up10d predictions: 1.129
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1647                8464
Actual Negative                 289               52804
Positive predictive power:
16.29%
Positive predictive accuracy:
85.07%
Confusi

In [31]:
# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

negmodel = PimaClassifier().to(device)
negcheckpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
negmodel.load_state_dict(negcheckpoint['model_state_dict'])
optimizer.load_state_dict(negcheckpoint['optimizer_state_dict'])
epoch = negcheckpoint['epoch']
loss = negcheckpoint['loss']
negmodel.eval()
negmodel = negmodel.to(device)

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(neg_predictions))
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 9up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 9up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 9up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 7up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 7up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 7up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 5up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 5up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 5up10d predictions:", mean_value)



63205
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1720                4205
Actual Negative                 532               56748
Positive predictive power:
29.03%
Positive predictive accuracy:
76.38%
Confusion Matrix for merged 9up10d predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1459                4466
Actual Negative                 351               56929
Positive predictive power for 9up10d:
24.62%
Positive predictive accuracy for 9up10d:
80.61%
Mean change for incorrect 9up10d predictions: 1.033
Mean change for correct 9up10d predictions: 1.126
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1686                8435
Actual Negative                 308               52776
Positive predictive power:
16.66%
Positive predictive accuracy:
84.55%
Confusi

In [4]:
# Ensure reproducibility
np.random.seed(42)

test_10d_change_file  = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_change10d_for_NN_20210514_to_20240404.tsv'
change10d_dataset = pd.read_csv(test_10d_change_file, delimiter='\t')
print(change10d_dataset.iloc[:5,:5])
print(change10d_dataset.shape)

neg_output_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_5down10d_for_NN_20210514_to_20240404.tsv'
neg_output_dataset = pd.read_csv(neg_output_file, delimiter='\t')
print(neg_output_dataset.iloc[:5,:5])
print(neg_output_dataset.shape)
print(neg_output_dataset.value_counts())

output9up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_9up10d_for_NN_20210514_to_20240404.tsv'
output9up10d_dataset = pd.read_csv(output9up10d_file, delimiter='\t')
print(output9up10d_dataset.iloc[:5,:5])
print(output9up10d_dataset.shape)
print(output9up10d_dataset.value_counts())

output7up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_7up10d_for_NN_20210514_to_20240404.tsv'
output7up10d_dataset = pd.read_csv(output7up10d_file, delimiter='\t')
print(output7up10d_dataset.iloc[:5,:5])
print(output7up10d_dataset.shape)
print(output7up10d_dataset.value_counts())

output5up10d_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_5up10d_for_NN_20210514_to_20240404.tsv'
output5up10d_dataset = pd.read_csv(output5up10d_file, delimiter='\t')
print(output5up10d_dataset.iloc[:5,:5])
print(output5up10d_dataset.shape)
print(output5up10d_dataset.value_counts())

# Shuffle column names
shuffled_columns = np.random.permutation(input_dataset.columns)

# Calculate split sizes
total_columns = len(input_dataset.columns)
train_size = int(0.8 * total_columns)
# The remaining columns will go to the test set

# Split the columns
train_columns = shuffled_columns[:train_size]
test_columns = shuffled_columns[train_size:]

# Create subsets of the DataFrame based on the columns
train_df = input_dataset[train_columns]
test_df = input_dataset[test_columns]

output9up10d_dataset.index = input_dataset.columns
output7up10d_dataset.index = input_dataset.columns
output5up10d_dataset.index = input_dataset.columns
neg_output_dataset.index = input_dataset.columns
change10d_dataset.index = input_dataset.columns

train9up10d_output = output9up10d_dataset.T[train_columns].T
test9up10d_output = output9up10d_dataset.T[test_columns].T

train7up10d_output = output7up10d_dataset.T[train_columns].T
test7up10d_output = output7up10d_dataset.T[test_columns].T

train5up10d_output = output5up10d_dataset.T[train_columns].T
test5up10d_output = output5up10d_dataset.T[test_columns].T

train_negoutput = neg_output_dataset.T[train_columns].T
test_negoutput = neg_output_dataset.T[test_columns].T

train_10dChange = change10d_dataset.T[train_columns].T
test_10dChange = change10d_dataset.T[test_columns].T

train9up10d_output.index = range(1, len(train9up10d_output) + 1)
test9up10d_output.index = range(1, len(test9up10d_output) + 1)
print(train9up10d_output.value_counts())
print(test9up10d_output.value_counts())

train7up10d_output.index = range(1, len(train7up10d_output) + 1)
test7up10d_output.index = range(1, len(test7up10d_output) + 1)
print(train7up10d_output.value_counts())
print(test7up10d_output.value_counts())

train5up10d_output.index = range(1, len(train5up10d_output) + 1)
test5up10d_output.index = range(1, len(test5up10d_output) + 1)
print(train5up10d_output.value_counts())
print(test5up10d_output.value_counts())

train_negoutput.index = range(1, len(train_negoutput) + 1)
test_negoutput.index = range(1, len(test_negoutput) + 1)
print(train_negoutput.value_counts())
print(test_negoutput.value_counts())

train_10dChange.index = range(1, len(train_10dChange) + 1)
test_10dChange.index = range(1, len(test_10dChange) + 1)


         x
1  1.02226
2  1.04186
3  1.04381
4  1.03341
5  1.03589
(316022, 1)
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    234112
1     81910
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    286620
1     29402
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    265159
1     50863
dtype: int64
   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    229013
1     87009
dtype: int64
x
0    229340
1     23477
dtype: int64
x
0    57280
1     5925
dtype: int64
x
0    212075
1     40742
dtype: int64
x
0    53084
1    10121
dtype: int64
x
0    183182
1     69635
dtype: int64
x
0    45831
1    17374
dtype: int64
x
0    187334
1     65483
dtype: int64
x
0    46778
1    16427
dtype: int64


In [ ]:
# Ensure reproducibility
np.random.seed(42)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_negoutput), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act_output(self.output(x))
        return x
 
negmodel = PimaClassifier().to(device)
print(negmodel)

pos_weight = torch.tensor([2.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(negmodel.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 2000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = negmodel(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': negmodel.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the negmodel
neg_predictions = (test_model(X) > 0.5).int().to('cpu')
neg_predictions = neg_predictions.numpy()
test_pred = pd.DataFrame(neg_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_negoutput
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for negative predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

neg_predictions = 1 - neg_predictions

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model9up10d = PimaClassifier().to(device)
print(model9up10d)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model9up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 2000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model9up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model9up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model7up10d = PimaClassifier().to(device)
print(model7up10d)

pos_weight = torch.tensor([2], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model7up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 2000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model7up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model7up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train9up10d_output), dtype=torch.float32).reshape(-1, 1)

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

model5up10d = PimaClassifier().to(device)
print(model5up10d)

pos_weight = torch.tensor([2.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model5up10d.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 2000
best_loss = float('inf')
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model5up10d(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    if loss < best_loss:
        best_loss = loss
        # Save model checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model5up10d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
        print("Saved improved model")

test_model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
test_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_model.eval()
with torch.no_grad():
    y_pred = test_model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (test_model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


Using device: cuda
PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)
Finished epoch 0, latest loss 0.9376556873321533
Saved improved model
Finished epoch 1, latest loss 0.9235866665840149
Saved improved model
Finished epoch 2, latest loss 0.9074278473854065
Saved improved model
Finished epoch 3, latest loss 0.8991867303848267
Saved improved model
Finished epoch 4, latest loss 0.8910913467407227
Saved improved model
Finished epoch 5, latest loss 0.8820620775222778
Saved improved model
Finished epoch 6, latest loss 0.8773768544197083
Saved improved model
Finished epoch 7, latest loss 0.8694850206375122
Saved improved model
Finished epoch 8, latest loss 0.8654044270515442
Saved improved model
Fi

In [8]:
# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

negmodel = PimaClassifier().to(device)
negcheckpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_negmodel.pth')
negmodel.load_state_dict(negcheckpoint['model_state_dict'])
optimizer.load_state_dict(negcheckpoint['optimizer_state_dict'])
epoch = negcheckpoint['epoch']
loss = negcheckpoint['loss']
negmodel.eval()
negmodel = negmodel.to(device)

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(neg_predictions))
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model9up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test9up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 9up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 9up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 9up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 9up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model7up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test7up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 7up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 7up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 7up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 7up10d predictions:", mean_value)

###

model = PimaClassifier().to(device)
checkpoint = torch.load('E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_best_model5up10d.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()
model = model.to(device)

input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)
input_tensor = input_tensor.to(device)
predictions = (model(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
predictions = predictions.numpy()

merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test5up10d_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged 5up10d predictions:")
print(confusion_matrix)
print("Positive predictive power for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy for 5up10d:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect 5up10d predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct 5up10d predictions:", mean_value)



63205
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1558                4367
Actual Negative                 410               56870
Positive predictive power:
26.3%
Positive predictive accuracy:
79.17%
Confusion Matrix for merged 9up10d predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1343                4582
Actual Negative                 285               56995
Positive predictive power for 9up10d:
22.67%
Positive predictive accuracy for 9up10d:
82.49%
Mean change for incorrect 9up10d predictions: 1.032
Mean change for correct 9up10d predictions: 1.128
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                2082                8039
Actual Negative                 293               52791
Positive predictive power:
20.57%
Positive predictive accuracy:
87.66%
Confusio

In [45]:
X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_negoutput), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        #self.hidden4 = nn.Linear(12, 4)
        #self.act4 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        #x = self.act4(self.hidden4(x))
        x = self.act_output(self.output(x))
        return x
 
negmodel = PimaClassifier().to(device)
print(negmodel)

pos_weight = torch.tensor([2.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(negmodel.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = negmodel(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = negmodel(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the negmodel
neg_predictions = (negmodel(X) > 0.5).int().to('cpu')
neg_predictions = neg_predictions.numpy()
test_pred = pd.DataFrame(neg_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_negoutput
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for negative predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

neg_predictions = 1 - neg_predictions
#print(neg_predictions)

X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_output), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        #self.hidden4 = nn.Linear(12, 4)
        #self.act4 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        #x = self.act4(self.hidden4(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier().to(device)
print(model)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

#print(predictions)

###

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)

Using device: cuda
PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)
Finished epoch 0, latest loss 0.9709123969078064
Finished epoch 1, latest loss 0.959958553314209
Finished epoch 2, latest loss 0.9461503028869629
Finished epoch 3, latest loss 0.9359431862831116
Finished epoch 4, latest loss 0.9327227473258972
Finished epoch 5, latest loss 0.922701895236969
Finished epoch 6, latest loss 0.9164071083068848
Finished epoch 7, latest loss 0.9102634787559509
Finished epoch 8, latest loss 0.9003651142120361
Finished epoch 9, latest loss 0.8953574895858765
Finished epoch 10, latest loss 0.9102886915206909
Finished epoch 11, latest loss 0.8989652395248413
Finished epoch 12, latest loss 0.89338940382

In [46]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


63204
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1605                4209
Actual Negative                 519               56871
Positive predictive power:
27.61%
Positive predictive accuracy:
75.56%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1379                4435
Actual Negative                 378               57012
Positive predictive power:
23.72%
Positive predictive accuracy:
78.49%
Mean change for incorrect predictions: 1.036
Mean change for correct predictions: 1.128


In [47]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


63205
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1606                4319
Actual Negative                 532               56748
Positive predictive power:
27.11%
Positive predictive accuracy:
75.12%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1368                4557
Actual Negative                 376               56904
Positive predictive power:
23.09%
Positive predictive accuracy:
78.44%
Mean change for incorrect predictions: 1.032
Mean change for correct predictions: 1.126


In [48]:
torch.save(model.state_dict(), "E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_9up10d_model.pt")

torch.save(negmodel.state_dict(), "E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_5down10d_model.pt")


In [49]:
# Ensure reproducibility
output_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_5up10d_for_NN_20210514_to_20240404.tsv'
output_dataset = pd.read_csv(output_file, delimiter='\t')
print(output_dataset.iloc[:5,:5])
print(output_dataset.shape)
print(output_dataset.value_counts())

output_dataset.index = input_dataset.columns

train_output = output_dataset.T[train_columns].T
cv_output = output_dataset.T[cv_columns].T
test_output = output_dataset.T[test_columns].T

train_output.index = range(1, len(train_output) + 1)
cv_output.index = range(1, len(cv_output) + 1)
test_output.index = range(1, len(test_output) + 1)


   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    229013
1     87009
dtype: int64


In [50]:
X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_output), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        #self.hidden4 = nn.Linear(12, 4)
        #self.act4 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        #x = self.act4(self.hidden4(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier().to(device)
print(model)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

#print(predictions)

###
neg_predictions = (negmodel(X) > 0.5).int().to('cpu')
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)



Using device: cuda
PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)
Finished epoch 0, latest loss 0.7778500318527222
Finished epoch 1, latest loss 0.772450864315033
Finished epoch 2, latest loss 0.7727213501930237
Finished epoch 3, latest loss 0.7684698104858398
Finished epoch 4, latest loss 0.7645014524459839
Finished epoch 5, latest loss 0.7624107003211975
Finished epoch 6, latest loss 0.7623335123062134
Finished epoch 7, latest loss 0.7573959231376648
Finished epoch 8, latest loss 0.7607902884483337
Finished epoch 9, latest loss 0.7545563578605652
Finished epoch 10, latest loss 0.7526525259017944
Finished epoch 11, latest loss 0.7537382245063782
Finished epoch 12, latest loss 0.7500900030

In [51]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)



63204
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                8149                9169
Actual Negative                2239               43647
Positive predictive power:
47.06%
Positive predictive accuracy:
78.45%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                7218               10100
Actual Negative                1618               44268
Positive predictive power:
41.68%
Positive predictive accuracy:
81.69%
Mean change for incorrect predictions: 1.005
Mean change for correct predictions: 1.082


In [52]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


63205
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                8209                9165
Actual Negative                2309               43522
Positive predictive power:
47.25%
Positive predictive accuracy:
78.05%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                7248               10126
Actual Negative                1638               44193
Positive predictive power:
41.72%
Positive predictive accuracy:
81.57%
Mean change for incorrect predictions: 1.006
Mean change for correct predictions: 1.081


In [53]:
torch.save(model.state_dict(), "E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_5up10d_model.pt")

In [54]:
# Ensure reproducibility
output_file = 'E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/test_output_7up10d_for_NN_20210514_to_20240404.tsv'
output_dataset = pd.read_csv(output_file, delimiter='\t')
print(output_dataset.iloc[:5,:5])
print(output_dataset.shape)
print(output_dataset.value_counts())

output_dataset.index = input_dataset.columns

train_output = output_dataset.T[train_columns].T
cv_output = output_dataset.T[cv_columns].T
test_output = output_dataset.T[test_columns].T

train_output.index = range(1, len(train_output) + 1)
cv_output.index = range(1, len(cv_output) + 1)
test_output.index = range(1, len(test_output) + 1)


   x
1  0
2  0
3  0
4  0
5  0
(316022, 1)
x
0    265159
1     50863
dtype: int64


In [55]:
X = torch.tensor(np.array(train_df.T), dtype=torch.float32)
y = torch.tensor(np.array(train_output), dtype=torch.float32).reshape(-1, 1)

# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move data to the chosen device
X = X.to(device)
y = y.to(device)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        #self.hidden4 = nn.Linear(12, 4)
        #self.act4 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        #x = self.act4(self.hidden4(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier().to(device)
print(model)

pos_weight = torch.tensor([1.5], device=device)  # Set your_pos_weight based on your needs
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 2000
batch_size = 1000
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')
    
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

# make class predictions with the model
predictions = (model(X) > 0.5).int().to('cpu')
predictions = predictions.numpy()

###

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

#print(predictions)

###
neg_predictions = (negmodel(X) > 0.5).int().to('cpu')
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions

merge_predictions = np.logical_and(neg_predictions, predictions)
merge_predictions = pd.DataFrame(merge_predictions,columns=['Predicted'])
merge_predictions.index = range(1, len(merge_predictions) + 1)
actual_values = train_output
actual_values.columns = ['Actual']

test = actual_values.join(merge_predictions)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(train_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)




Using device: cuda
PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)
Finished epoch 0, latest loss 0.7468140125274658
Finished epoch 1, latest loss 0.748431921005249
Finished epoch 2, latest loss 0.7484720945358276
Finished epoch 3, latest loss 0.7478343844413757
Finished epoch 4, latest loss 0.7464306950569153
Finished epoch 5, latest loss 0.7461487650871277
Finished epoch 6, latest loss 0.7465912103652954
Finished epoch 7, latest loss 0.7451706528663635
Finished epoch 8, latest loss 0.7429878115653992
Finished epoch 9, latest loss 0.7421632409095764
Finished epoch 10, latest loss 0.7442635893821716
Finished epoch 11, latest loss 0.7422481179237366
Finished epoch 12, latest loss 0.7377665638

In [56]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(cv_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)



63204
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                3628                6483
Actual Negative                1123               51970
Positive predictive power:
35.88%
Positive predictive accuracy:
76.36%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                3209                6902
Actual Negative                 852               52241
Positive predictive power:
31.74%
Positive predictive accuracy:
79.02%
Mean change for incorrect predictions: 1.02
Mean change for correct predictions: 1.105


In [57]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(test_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = test_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


63205
63205
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                3667                6454
Actual Negative                1114               51970
Positive predictive power:
36.23%
Positive predictive accuracy:
76.7%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                3239                6882
Actual Negative                 822               52262
Positive predictive power:
32.0%
Positive predictive accuracy:
79.76%
Mean change for incorrect predictions: 1.019
Mean change for correct predictions: 1.104


In [58]:
torch.save(model.state_dict(), "E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_7up10d_model.pt")

         x
1  1.02226
2  1.04186
3  1.04381
4  1.03341
5  1.03589
(316022, 1)
Mean of selected rows: 1.0035831841302136


In [63]:
# Check for CUDA and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(train_df.shape[0], 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 12)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(12, 8)
        self.act3 = nn.ReLU()
        #self.hidden4 = nn.Linear(12, 4)
        #self.act4 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        #x = self.act4(self.hidden4(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier().to(device)
model.load_state_dict(torch.load("E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_9up10d_model.pt"))
model.eval()

negmodel = PimaClassifier().to(device)
negmodel.load_state_dict(torch.load("E:/Market_Data/DiscountOptionData/DTNSubscription/revised_derived_aggregates/best_5down10d_model.pt"))
negmodel.eval()

Using device: cuda


PimaClassifier(
  (hidden1): Linear(in_features=1616, out_features=64, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=64, out_features=12, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=12, out_features=8, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)

In [69]:
# Step 1: Set the model to evaluation mode
negmodel.eval()
model.eval()

# Step 2: Prepare your input data (assuming new_input is your raw data)
# For example, if your model expects input size of (1, C, H, W)
input_tensor = torch.tensor(np.array(cv_df.T), dtype=torch.float32)

# Step 3: Move your model and input data to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
negmodel = negmodel.to(device)
input_tensor = input_tensor.to(device)

predictions = (model(input_tensor) > 0.5).int().to('cpu')
neg_predictions = (negmodel(input_tensor) > 0.5).int().to('cpu')
print(len(predictions))
print(len(neg_predictions))
predictions = predictions.numpy()
neg_predictions = neg_predictions.numpy()
neg_predictions = 1 - neg_predictions
merge_predictions = np.logical_and(neg_predictions, predictions)

test_pred = pd.DataFrame(predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for positive predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

test_pred = pd.DataFrame(merge_predictions,columns=['Predicted'])
test_pred.index = range(1, len(test_pred) + 1)

actual_values = cv_output
actual_values.columns = ['Actual']

test = actual_values.join(test_pred)

TP = ((test['Actual'] == 1) & (test['Predicted'] == 1)).sum()
TN = ((test['Actual'] == 0) & (test['Predicted'] == 0)).sum()
FP = ((test['Actual'] == 0) & (test['Predicted'] == 1)).sum()
FN = ((test['Actual'] == 1) & (test['Predicted'] == 0)).sum()

# Creating a DataFrame for the confusion matrix
confusion_matrix = pd.DataFrame([[TP, FN], [FP, TN]],
                                columns=['Predicted Positive', 'Predicted Negative'],
                                index=['Actual Positive', 'Actual Negative'])

print("Confusion Matrix for merged predictions:")
print(confusion_matrix)
print("Positive predictive power:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[0,1]))*100
print(str(round(test_val,2)) + "%")
print("Positive predictive accuracy:")
test_val = (confusion_matrix.iloc[0,0] / (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,0]))*100
print(str(round(test_val,2)) + "%")

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 0) & (test['Predicted'] == 1)), dtype=bool)
print(test.shape)
print(test_10dChange.shape)
print(cv_df.shape)
print(cv_output.shape)
# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for incorrect predictions:", mean_value)

# Convert the list to a boolean array
mask = np.array(((test['Actual'] == 1) & (test['Predicted'] == 1)), dtype=bool)

# Use the mask to filter rows and calculate the mean
mean_value = round(test_10dChange.loc[mask,'x'].mean(),3)

print("Mean change for correct predictions:", mean_value)


63204
63204
Confusion Matrix for positive predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1854                8257
Actual Negative                 270               52823
Positive predictive power:
18.34%
Positive predictive accuracy:
87.29%
Confusion Matrix for merged predictions:
                 Predicted Positive  Predicted Negative
Actual Positive                1582                8529
Actual Negative                 175               52918
Positive predictive power:
15.65%
Positive predictive accuracy:
90.04%
(63204, 2)
(63205, 1)
(1616, 63204)
(63204, 1)


IndexError: Boolean index has wrong length: 63204 instead of 63205